## OpenAlex Topic-Based Preprint Harvesting Pipeline

This notebook collects and stores large-scale database from the OpenAlex API by:
- Harvesting works for a predefined set of OpenAlex research topics, as given by topic_ids in excel sheets  
- Fetching records in parallel with retry logic and rate-limit handling  
- Extracting work metadata, topic assignments, etc.

**Input:**
- OpenAlex API (`/works`, `/topics`)
- Topic IDs excel (e.g., Psychology-related OpenAlex topics)
- Publication date range (`start_date`, `end_date`)

**Output Database:**
Field-based OpenAlex database, eg: `psychology_AI.db` 


In [1]:
import concurrent.futures
import threading

# Create a lock to control database access
db_lock = threading.Lock()
# Set number of concurrent threads (5 is usually safe for OpenAlex without hitting limits too hard)
MAX_WORKERS = 5

In [ ]:
import requests
import duckdb
import time
import json
from datetime import datetime, timedelta, date
from typing import List, Dict, Any, Optional
import pandas as pd
import os
from tqdm import tqdm

#Configs
OPENALEX_BASE = "https://api.openalex.org/works"
TOPIC_BASE = "https://api.openalex.org/topics"
PAGE_SIZE = 200
COUNT_LIMIT = 5000 
DB_DIR = './data/dbs'
if not os.path.exists(DB_DIR):
    os.makedirs(DB_DIR)
DB_FILENAME = 'openAlex_psychology.db'
DB_PATH = os.path.join(DB_DIR, DB_FILENAME)

#Safety Settings
MAX_RETRIES_PER_INTERVAL = 3   # Try each interval 3 times before giving up
MAX_CONSECUTIVE_FAILURES = 10  # If 10 intervals fail in a row, STOP the script.
consecutive_failure_count = 0  # Global counter (don't edit this manually)
failure_lock = threading.Lock() # Lock to update the counter safely

# Connect to DuckDB
conn = duckdb.connect(DB_PATH)
print(f"Connected to database at {DB_PATH}")

# Request Wrapper
def request_with_retry(url, params=None):
    """
    Makes a GET request. If it fails due to network issues or 
    server errors (500, 502, 503, 429), it waits and retries indefinitely.
    Raises exception only for client errors (400, 401, 404, etc).
    """
    delay = 2  # Start with 2 seconds delay
    while True:
        try:
            r = requests.get(url, params=params, timeout=30)
            
            # If successful, return response
            if r.status_code == 200:
                return r
            
            # If server error or rate limit, print and retry
            if r.status_code in [429, 500, 502, 503, 504]:
                print(f"Status {r.status_code} received. Retrying in {delay}s...")
                time.sleep(delay)
                delay = min(delay * 2, 60) # Exponential backoff up to 60s
                continue
            
            # If it's a client error (e.g. 404), raise it immediately
            r.raise_for_status()
            
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}. Retrying in {delay}s...")
            time.sleep(delay)
            delay = min(delay * 2, 60)

Connected to database at dbs/openAlex_psychology.db


In [3]:
def init_schema(conn):
    # conn.execute("DROP TABLE IF EXISTS works") 
    
    conn.execute("""
        CREATE TABLE IF NOT EXISTS works (
            id VARCHAR PRIMARY KEY,
            title VARCHAR,
            doi VARCHAR,
            publication_date DATE,
            primary_topic VARCHAR,
            version VARCHAR,
            fwci DOUBLE,
            citation_count INTEGER,
            mag_id BIGINT             -- New Column for MAG ID
        )
    """)
    # ... rest of the schema tables (topics, work_topics, harvest_intervals) remain the same ...
    conn.execute("""
        CREATE TABLE IF NOT EXISTS topics (
            topic_id VARCHAR PRIMARY KEY,
            topic_name VARCHAR UNIQUE
        )
    """)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS work_topics (
            work_id VARCHAR,
            topic_id VARCHAR,
            score DOUBLE,
            PRIMARY KEY (work_id, topic_id),
            FOREIGN KEY (work_id) REFERENCES works(id),
            FOREIGN KEY (topic_id) REFERENCES topics(topic_id)
        )
    """)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS harvest_intervals (
            topic_id VARCHAR,
            start_date DATE,
            end_date DATE,
            last_token VARCHAR,
            fetched_count INTEGER DEFAULT 0,
            status VARCHAR,
            last_updated TIMESTAMP,
            PRIMARY KEY (topic_id, start_date, end_date)
        )
    """)
    print("Schema initialized with MAG ID.")

init_schema(conn)

Schema initialized with MAG ID.


In [ ]:
def get_count(topic_id, start_date, end_date):
    params = {
        "filter": f"primary_topic.id:{topic_id},from_publication_date:{start_date},to_publication_date:{end_date},type:preprint|article",
        "per-page": 1
    }
    # Uses the retry wrapper
    r = request_with_retry(OPENALEX_BASE, params=params)
    return int(r.json()["meta"]["count"])

def get_topic_name(topic_id):
    # Uses the retry wrapper
    r = request_with_retry(f"{TOPIC_BASE}/{topic_id}")
    data = r.json()
    return data.get("display_name") or data.get("name")

def fetch_page(topic_id, start_date, end_date, cursor="*"):
    params = {
        "filter": f"primary_topic.id:{topic_id},from_publication_date:{start_date},to_publication_date:{end_date},type:preprint|article",
        "per-page": PAGE_SIZE,
        "cursor": cursor
    }
    # Uses the retry wrapper
    r = request_with_retry(OPENALEX_BASE, params=params)
    data = r.json()
    results = data.get("results", [])
    next_cursor = data["meta"].get("next_cursor")
    return results, next_cursor

In [5]:
def register_intervals(conn, topic_id, intervals):
    for start, end in intervals:
        conn.execute("""
            INSERT OR IGNORE INTO harvest_intervals
            (topic_id, start_date, end_date, status, fetched_count, last_updated)
            VALUES (?, ?, ?, 'pending', 0, NOW())
        """, [topic_id, start, end])

In [ ]:
def insert_works_bulk(conn, works_list):
    """
    Inserts a large list of works efficiently.
    """
    if not works_list:
        return
    
    # specific optimization: Convert date objects to string to ensure DuckDB compatibility
    for w in works_list:
        if isinstance(w.get('publication_date'), (date, datetime)):
            w['publication_date'] = str(w['publication_date'])

    df = pd.DataFrame(works_list)
    
    # We use the lock only for the split second of writing
    with db_lock:
        conn.register("batch_works", df)
        # Bulk insert handling duplicates automatically
        conn.execute("INSERT OR IGNORE INTO works SELECT * FROM batch_works")
        conn.unregister("batch_works")

def insert_topics_bulk(conn, results_list):
    """
    Extracts all topics and relations from the results list and bulk inserts them.
    Drastically faster than row-by-row checks.
    """
    if not results_list:
        return

    # 1. Prepare Data in Memory (No Lock needed yet)
    unique_topics = {}
    work_topic_relations = []

    for w in results_list:
        work_id = w.get("id").split('/')[-1]
        all_topics = w.get("topics") or []
        
        for t in all_topics:
            topic_id = t.get("id").split('/')[-1]
            topic_name = t.get("display_name") or t.get("name")
            score = t.get("score") or 0.0
            
            if topic_id and topic_name:
                unique_topics[topic_id] = topic_name
                work_topic_relations.append({
                    "work_id": work_id,
                    "topic_id": topic_id,
                    "score": float(score)
                })

    if not unique_topics:
        return

    # Convert to DataFrames
    topics_df = pd.DataFrame(list(unique_topics.items()), columns=["topic_id", "topic_name"])
    relations_df = pd.DataFrame(work_topic_relations)

    # 2. Bulk Write (Lock needed now)
    with db_lock:
        # A. Insert missing Topics
        # We assume existing topics are relatively static, but let's just use INSERT OR IGNORE
        conn.register("batch_topics", topics_df)
        conn.execute("INSERT OR IGNORE INTO topics SELECT * FROM batch_topics")
        conn.unregister("batch_topics")

        # B. Insert Work-Topic Relations
        if not relations_df.empty:
            conn.register("batch_relations", relations_df)
            conn.execute("INSERT OR IGNORE INTO work_topics SELECT * FROM batch_relations")
            conn.unregister("batch_relations")

def mark_interval_complete(conn, topic_id, start_date, end_date, total_count):
    """
    Updates the interval status to 'done' in one go.
    """
    with db_lock:
        conn.execute("""
            UPDATE harvest_intervals
            SET status='done', fetched_count=?, last_updated=now()
            WHERE topic_id=? AND start_date=? AND end_date=?
        """, [total_count, topic_id, start_date, end_date])

In [7]:
def fetch_interval_atomic(conn, topic_id, start_date, end_date):
    """
    Fetches the ENTIRE interval into memory first, then dumps to DB.
    Eliminates database locking during the download phase.
    """
    #Memory Buffers
    all_works = []
    all_raw_results = [] # We need raw results for topic parsing
    
    cursor = "*"
    finished = False
    
    # Download Loop
    while not finished:
        # If an interval was halfway done, we restart it to ensure consistency.
        
        results, next_cursor = fetch_page(topic_id, start_date, end_date, cursor)
        
        # Buffer raw results for topic extraction later
        all_raw_results.extend(results)

        # drop unused JSON fields
        for w in results:
            primary_topic = w.get("primary_topic", {}).get("display_name") if w.get("primary_topic") else None
            mag_id = w.get("ids", {}).get("mag")
            
            all_works.append({
                "id": w.get("id").split('/')[-1],
                "title": w.get("title"),
                "doi": w.get("doi"),
                "publication_date": w.get("publication_date"),
                "primary_topic": primary_topic,
                "version": w.get("primary_location", {}).get("version", {}),
                "fwci": w.get("fwci"),
                "citation_count": w.get("cited_by_count"),
                "mag_id": mag_id
            })

        if next_cursor is None:
            finished = True
        else:
            cursor = next_cursor

   
    if all_works:
        insert_works_bulk(conn, all_works)
        insert_topics_bulk(conn, all_raw_results)
        mark_interval_complete(conn, topic_id, start_date, end_date, len(all_works))

In [8]:
def split_range_safely(topic_id, start_date, end_date, limit=COUNT_LIMIT):
    """
    Recursively splits date ranges until the count is below the limit.
    """
    cnt = get_count(topic_id, start_date, end_date)
    print(f"   Checking range {start_date} -> {end_date}: count = {cnt}")
    
    if cnt <= limit:
        return [(start_date, end_date)]
    
    days = (end_date - start_date).days
    if days <= 1:
        # If we are down to 1 day and still over limit, we must accept it (or handle pagination)
        return [(start_date, end_date)]
        
    mid = start_date + timedelta(days=days // 2)
    left = split_range_safely(topic_id, start_date, mid, limit)
    right = split_range_safely(topic_id, mid + timedelta(days=1), end_date, limit)
    return left + right

def process_interval_wrapper(args):
    """
    Wrapper that retries the interval if it fails, and triggers a stop 
    if too many failures happen globally.
    """
    conn, topic_id, start_date, end_date = args
    global consecutive_failure_count

    # Check Circuit Breaker
    if consecutive_failure_count >= MAX_CONSECUTIVE_FAILURES:
        return (start_date, end_date, False)

    # Retry Loop for this specific interval
    for attempt in range(1, MAX_RETRIES_PER_INTERVAL + 1):
        try:
            # Try to fetch
            fetch_interval_atomic(conn, topic_id, start_date, end_date)
            
            # If successful:
            with failure_lock:
                consecutive_failure_count = 0 # Reset global failure count on success
            return (start_date, end_date, True)

        except Exception as e:
            print(f" Error in interval {start_date}->{end_date} (Attempt {attempt}/{MAX_RETRIES_PER_INTERVAL}): {e}")
            time.sleep(2 * attempt) # Wait a bit before retrying (2s, 4s, 6s)
    
    # If we failed 3 times, mark as a failure
    print(f" GIVING UP on interval {start_date}->{end_date} after {MAX_RETRIES_PER_INTERVAL} attempts.")
    with failure_lock:
        consecutive_failure_count += 1
        if consecutive_failure_count >= MAX_CONSECUTIVE_FAILURES:
            print(f"\n {consecutive_failure_count} consecutive failures detected. Stopping script to prevent skipping all intervals.\n")
            
    return (start_date, end_date, False)

def harvest_topics(conn, topic_ids, start_date, end_date):
    global consecutive_failure_count
    
    for topic_id in topic_ids:
        if not str(topic_id).startswith('t'):
             topic_id_fmt = 't' + str(topic_id)
        else:
             topic_id_fmt = str(topic_id)
             
        try:
            topic_name = get_topic_name(topic_id_fmt)
        except Exception as e:
            print(f"Could not fetch name for {topic_id_fmt}, skipping. Error: {e}")
            continue

        print(f"\n=== Harvesting topic {topic_id_fmt} ({topic_name}) ===")

        with db_lock:
            existing_intervals = conn.execute("""
                SELECT start_date, end_date 
                FROM harvest_intervals 
                WHERE topic_id = ? 
                ORDER BY start_date
            """, [topic_id_fmt]).fetchall()

        if existing_intervals:
            print(f" Found {len(existing_intervals)} existing intervals.")
            intervals = existing_intervals
        else:
            print(f" Calculating splits via API...")
            intervals = split_range_safely(topic_id_fmt, start_date, end_date)
            with db_lock:
                register_intervals(conn, topic_id_fmt, intervals)

        # Identify Pending Work
        tasks = []
        for safe_start, safe_end in intervals:
            # (Date fixing code same as before...)
            if hasattr(safe_start, 'date'): safe_start = safe_start.date()
            if hasattr(safe_end, 'date'): safe_end = safe_end.date()
            if isinstance(safe_start, str): safe_start = datetime.strptime(safe_start, '%Y-%m-%d').date()
            if isinstance(safe_end, str): safe_end = datetime.strptime(safe_end, '%Y-%m-%d').date()

            with db_lock:
                row = conn.execute("""
                    SELECT status FROM harvest_intervals
                    WHERE topic_id=? AND start_date=? AND end_date=?
                """, [topic_id_fmt, safe_start, safe_end]).fetchone()
            
            if row and row[0] == "done":
                continue
                
            tasks.append((conn, topic_id_fmt, safe_start, safe_end))

        print(f" Starting {len(tasks)} pending intervals with {MAX_WORKERS} threads...")

        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # Submit tasks
            futures = []
            for t in tasks:
                # Check stop signal BEFORE submitting
                if consecutive_failure_count >= MAX_CONSECUTIVE_FAILURES:
                    print(" Circuit breaker active. Cancelling remaining tasks.")
                    break
                futures.append(executor.submit(process_interval_wrapper, t))
            
            # Monitor results
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
                start, end, success = future.result()
                if not success and consecutive_failure_count >= MAX_CONSECUTIVE_FAILURES:
                    print(" Stopping execution loop due to critical failures.")
                    executor.shutdown(wait=False) # Force stop
                    return # Exit the function completely

        if consecutive_failure_count >= MAX_CONSECUTIVE_FAILURES:
            print(" Execution halted.")
            break

        print(f"=== Completed harvesting topic {topic_id_fmt} ===\n")

In [15]:
# Read topic IDs from an Excel file (must contain a 'topic_id' column)
topics_df = pd.read_excel("./data/topic_sheets/Psychology.xlsx")

# Extract topic IDs as a list (strip leading 't' if present)
topic_ids = (
    topics_df["topic_id"]
    .astype(str)
    .tolist()
)

start_date = date(2000, 1, 1)
end_date = date(2026, 1, 1)

harvest_topics(conn, topic_ids, start_date, end_date)



=== Harvesting topic t11084 (Workplace Health and Well-being) ===
 Calculating splits via API...
   Checking range 2000-01-01 -> 2026-01-01: count = 710
 Starting 1 pending intervals with 5 threads...


  0%|          | 0/1 [00:30<?, ?it/s]


KeyboardInterrupt: 